### Cargue de datos y vectorización, tokenizacion

In [107]:
import torch
import json
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
import pandas as pd
# Tokenizador BERT preentrenado en español
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

# Modelo BERT preentrenado en español
model = BertModel.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')

df_train= pd.read_csv('./Insumos/train.csv')


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
def embedding(texto):
  # Tokenizar y obtener embeddings para los textos
  tokens = tokenizer.encode(texto, add_special_tokens=True)
  input_ids = torch.tensor([tokens])
  with torch.no_grad():
      outputs = model(input_ids)
  # Obtener los embeddings de los tokens [CLS]
  embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
  return embedding

In [109]:
df_train['query_embedding']=df_train['query'].apply(lambda x: embedding(x))

In [110]:
df_train['document_embedding']=df_train['document'].apply(lambda x: embedding(x))

In [114]:
# Busqueda del documento con mayor simulitud y ranqueo
def search_docs_onprimse(df, user_query, top_n=3, to_print=True):
    embedding = df.query(f'query=="{user_query}"')\
                    [['query','query_embedding']].drop_duplicates(subset='query')\
                    .query_embedding.values[0]
    df["similarities"] = df.document_embedding.apply(lambda x: F.cosine_similarity(x, embedding,dim=0))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

In [115]:
df_resuelto= pd.DataFrame()
preguntas={}
mmr=[]
for query in df_train['query'].unique():
    respuestaTemporal= search_docs_onprimse(df_train,query, to_print=False)
    documentos= respuestaTemporal.document.to_list()
    preguntas[query]= documentos
    try:
        valor= respuestaTemporal.reset_index(drop=True)\
                .query('label==1').index[0]+1
        mmrTemporal= 1/valor
    except:
        mmrTemporal=0
    mmr.append(mmrTemporal)

In [113]:
# Especificamos la ruta del archivo JSON
ruta_archivo = "./Ejecuciones/results_train_ada_v2.json"

# Guardamos el diccionario como JSON en el archivo
with open(ruta_archivo,"w") as archivo:
    json.dump(preguntas, archivo)

print("Resultados guardados en results.json")
print('La media de los MMR es :',sum(mmr)/len(mmr))

Resultados guardados en results.json
La media de los MMR es : 0.33875
